学习资料链接：：https://blog.csdn.net/xindoo/article/details/134483093

重点总结：
1. 就在上周（20231106），OpenAI在开发者大会上（openai第一届开发者大会），升级了函数调用的功能，在新的gpt-3.5和gpt-4模型中，可以在单次对话中调用多个函数了，而且在python SDK中也提供了并发函数调用相关的接口，无疑这将大幅减少大语言模型和现实世界之间交互的开发复杂度；
2. 目前只有gpt-4-1106-preview和gpt-3.5-turbo-1106两个模式支持单词对话同时调用多个模型的，其他模型均不支持。
3. openAI改变了api中传递function的参数，废弃了 functions和 function_call，改用了tools和tool_choice两个新参数，我猜测是为了未来增加更多的工具支持。


# 案例

  输出的结果是根据百度、谷歌和必应三个搜索引擎的结果，'xindoo'可能是一个技术博主、后端工程师以及Python爱好者。


In [6]:
import openai
from openai import OpenAI
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

from pathlib import Path
import json


In [7]:
def search_baidu(keyword):
    """从百度搜索引擎中搜索关键词"""
    return f"{keyword}是一个技术博主"

def search_google(keyword):
    """从谷歌搜索引擎中搜索关键词"""
    return f"{keyword}是一个后端工程师"

def search_bing(keyword):
    """从必应搜索引擎中搜索关键词"""
    return f"{keyword}是一个Python爱好者"


In [8]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_baidu",
            "description": "从百度搜索引擎中搜索关键词",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "搜索关键词",
                    }
                },
                "required": ["keyword"],
            },
        }
    },    
    {
        "type": "function",
        "function": {
            "name": "search_google",
            "description": "从google搜索引擎中搜索关键词",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "搜索关键词",
                    }
                },
                "required": ["keyword"],
            },
        }
    },        
    {
        "type": "function",
        "function": {
            "name": "search_bing",
            "description": "从bing搜索引擎中搜索关键词",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "搜索关键词",
                    }
                },
                "required": ["keyword"],
            },
        }
    }
]
available_functions = { "search_baidu": search_baidu, "search_google": search_google, "search_bing": search_bing } 


In [9]:
client = OpenAI()

In [15]:
def search(keyword):
    messages = [{"role": "user", "content": f"汇总下百度、谷歌、必应三个搜索引擎关于'{keyword}'的结果"}]

    # 发起首次请求，告诉gpt要做什么，已经有哪些函数可以调动 
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto", 
    )
    print("=====response 类型:", type(response))
    print("=====response :", response)
    response_message = response.choices[0].message
    print("=====response_message 类型:", type(response_message))
    print("=====response_message :", response_message)
    tool_calls = response_message.tool_calls
    print("=====tool_calls 类型:", type(tool_calls))
    print("=====tool_calls :", tool_calls)

    if tool_calls:
        # 解析所有需要调用的函数及参数
        messages.append(response_message)  # 注意这里要将openai的回复也拼接到消息列表里
        print("=====messages 类型:", type(messages))
        print("=====messages :", messages)
        # 将所有函数调用的结果拼接到消息列表里
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args) 
            print(function_response)

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )

        return second_response.choices[0].message.content
    
    




In [16]:
print(search("xindoo"))


=====response 类型: <class 'openai.types.chat.chat_completion.ChatCompletion'>
=====response : ChatCompletion(id='chatcmpl-8NB6XLV10K7w45GQavNodSQn8q5Vh', choices=[Choice(finish_reason='tool_calls', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_B23AJ0twhnJ1WjOTFLQ6yXTl', function=Function(arguments='{"keyword": "xindoo"}', name='search_baidu'), type='function'), ChatCompletionMessageToolCall(id='call_5AbgdgVXl7PHgK2shAnnxjwn', function=Function(arguments='{"keyword": "xindoo"}', name='search_google'), type='function'), ChatCompletionMessageToolCall(id='call_0U2Ub8OovrfJXS0t8Chb3MUP', function=Function(arguments='{"keyword": "xindoo"}', name='search_bing'), type='function')]))], created=1700534921, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_eeff13170a', usage=CompletionUsage(completion_tokens=65, prompt_tokens=175, total_tokens=240))
=====response_message 类型: <